<a href="https://colab.research.google.com/github/harirm85/Hari_ML_workshop/blob/main/EDA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy the Jason file and set security
!mkdir -p ~/.kaggle
!cp '/content/drive/MyDrive/Kaggle/kaggle.json' ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

#Download data
!kaggle competitions download -c home-credit-default-risk

import zipfile

# Specify the path to the downloaded zip file
zip_path = "/content/home-credit-default-risk.zip"

# Specify the directory to extract the files to
extract_path = "/content/home-credit-default-risk"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Mounted at /content/drive
 98% 671M/688M [00:08<00:00, 83.6MB/s]
100% 688M/688M [00:08<00:00, 84.6MB/s]


In [3]:
#Take app train data and load to DF
app_data = pd.read_csv('/content/home-credit-default-risk/application_train.csv')
bur_data = pd.read_csv('/content/home-credit-default-risk/bureau.csv')

print(bur_data.shape)

(1716428, 17)


In [4]:
bur_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SK_ID_CURR              1716428 non-null  int64  
 1   SK_ID_BUREAU            1716428 non-null  int64  
 2   CREDIT_ACTIVE           1716428 non-null  object 
 3   CREDIT_CURRENCY         1716428 non-null  object 
 4   DAYS_CREDIT             1716428 non-null  int64  
 5   CREDIT_DAY_OVERDUE      1716428 non-null  int64  
 6   DAYS_CREDIT_ENDDATE     1610875 non-null  float64
 7   DAYS_ENDDATE_FACT       1082775 non-null  float64
 8   AMT_CREDIT_MAX_OVERDUE  591940 non-null   float64
 9   CNT_CREDIT_PROLONG      1716428 non-null  int64  
 10  AMT_CREDIT_SUM          1716415 non-null  float64
 11  AMT_CREDIT_SUM_DEBT     1458759 non-null  float64
 12  AMT_CREDIT_SUM_LIMIT    1124648 non-null  float64
 13  AMT_CREDIT_SUM_OVERDUE  1716428 non-null  float64
 14  CR

In [ ]:
bur_data[bur_data['DAYS_ENDDATE_FACT']<0]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.00,-153.00,NaN,0,"91,323.00",0.00,NaN,0.00,Consumer credit,-131,NaN
7,162297,5714469,Closed,currency 1,-1896,0,"-1,684.00","-1,710.00","14,985.00",0,"76,878.45",0.00,0.00,0.00,Consumer credit,-1710,NaN
8,162297,5714470,Closed,currency 1,-1146,0,-811.00,-840.00,0.00,0,"103,007.70",0.00,0.00,0.00,Consumer credit,-840,NaN
11,162297,5714473,Closed,currency 1,-2456,0,-629.00,-825.00,NaN,0,"675,000.00",0.00,0.00,0.00,Consumer credit,-706,NaN
14,238881,5714482,Closed,currency 1,-318,0,-187.00,-187.00,NaN,0,0.00,0.00,0.00,0.00,Credit card,-185,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716422,375755,5057742,Closed,currency 1,-2648,0,"31,129.00",-189.00,NaN,0,"202,500.00",0.00,NaN,0.00,Credit card,-109,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,"-2,433.00","-2,493.00","5,476.50",0,"38,130.84",0.00,0.00,0.00,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,"-1,628.00",-970.00,NaN,0,"15,570.00",NaN,NaN,0.00,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,"-1,513.00","-1,513.00",NaN,0,"36,000.00",0.00,0.00,0.00,Consumer credit,-1508,NaN


In [6]:
bur_data['AMT_ANNUITY'].value_counts()

0.00          256915
4,500.00        5182
13,500.00       3147
22,500.00       2502
9,000.00        1725
               ...  
21,228.75          1
7,263.27           1
49,215.01          1
144,405.00         1
24,351.44          1
Name: AMT_ANNUITY, Length: 40321, dtype: int64

In [9]:
from pandas.core.dtypes.cast import Sized
#bur_data1 = bur_data[bur_data['SK_ID_CURR']==282297]

Summary_data = bur_data.groupby('SK_ID_CURR', as_index=False).agg(
    BUR_CNT = ("SK_ID_BUREAU","count")
    , BUR_ACTIVE_CNT = ("CREDIT_ACTIVE", lambda x: sum(x=='Active'))
    , BUR_SOLD_CNT = ("CREDIT_ACTIVE", lambda x: sum(x=='Sold'))
    , BUR_BAD_DEPT_CNT = ("CREDIT_ACTIVE", lambda x: sum(x=='Bad debt'))
    , BUR_CUR_USED = ("CREDIT_CURRENCY","nunique")
    , BUR_CRD_TYPES = ("CREDIT_TYPE","nunique")
    # , BUR_CRD_TYPES_OTHR = ("CREDIT_TYPE", lambda x: sum(x) if x in ['Consumer cedit','Credit card','Car loan','Mortgage'] else 0)

    , BUR_DAYS_CREDIT_VAR = ("DAYS_CREDIT", lambda x:max(x) - min(x))
    , BUR_OD_DAY_CNT = ("CREDIT_DAY_OVERDUE", lambda x:sum(x>0))
    , BUR_OD_DAY_VAR = ("CREDIT_DAY_OVERDUE", lambda x:max(x) - min(x))
    , BUR_DAYS_ED_FACT_VAR = ("DAYS_ENDDATE_FACT", lambda x:max(x) - min(x))
    , BUR_OD_AMT_MAX_CNT = ("AMT_CREDIT_MAX_OVERDUE", lambda x:sum(x>0))
    , BUR_OD_AMT_MAX_VAR = ("AMT_CREDIT_MAX_OVERDUE", lambda x:max(x) - min(x))
    , BUR_CD_PRO_CNT = ("CNT_CREDIT_PROLONG", lambda x:sum(x>0))
    , BUR_CD_PRO_VAR = ("CNT_CREDIT_PROLONG", lambda x:max(x) - min(x))
    , BUR_CD_AMT_VAR = ("AMT_CREDIT_SUM", lambda x:max(x) - min(x))
    , BUR_CD_DBT_CNT = ("AMT_CREDIT_SUM_DEBT", lambda x:sum(x>0))
    , BUR_CD_DBT_VAR = ("AMT_CREDIT_SUM_DEBT", lambda x:max(x) - min(x))
    , BUR_CD_LMT_CNT = ("AMT_CREDIT_SUM_LIMIT", lambda x:sum(x>0))
    , BUR_CD_LMT_VAR = ("AMT_CREDIT_SUM_LIMIT", lambda x:max(x) - min(x))
    , BUR_OD_AMT_SUM_CNT = ("AMT_CREDIT_SUM_OVERDUE", lambda x:sum(x>0))
    , BUR_OD_AMT_SUM_VAR = ("AMT_CREDIT_SUM_OVERDUE", lambda x:max(x) - min(x))
    , BUR_DAYS_CREDIT_UPDT_VAR = ("DAYS_CREDIT_UPDATE", lambda x:max(x) - min(x))
    , BUR_ANNUITY_CNT = ("AMT_CREDIT_SUM_DEBT", lambda x:sum(x>0))
    , BUR_ANNUITY_VAR = ("AMT_CREDIT_SUM_DEBT", lambda x:max(x) - min(x))
)
Summary_data.shape

(305811, 25)

In [8]:
Summary_data

,SK_ID_CURR,BUR_CNT,BUR_ACTIVE_CNT,BUR_SOLD_CNT,BUR_BAD_DEPT_CNT,BUR_CUR_USED,BUR_CRD_TYPES,BUR_DAYS_CREDIT_VAR,BUR_OD_DAY_CNT,BUR_OD_DAY_VAR,...,BUR_CD_AMT_VAR,BUR_CD_DBT_CNT,BUR_CD_DBT_VAR,BUR_CD_LMT_CNT,BUR_CD_LMT_VAR,BUR_OD_AMT_SUM_CNT,BUR_OD_AMT_SUM_VAR,BUR_DAYS_CREDIT_UPDT_VAR,BUR_ANNUITY_CNT,BUR_ANNUITY_VAR
0,282297,11,5,0,0,1,3,2663,1,2603,...,"910,944.00",4,"435,573.00",1,"59,083.79",1,231.53,2148,4,"435,573.00"


In [16]:
combined_data=app_data[['SK_ID_CURR','TARGET']].join(Summary_data,on='SK_ID_CURR',lsuffix='_APP', rsuffix='_BUR')
combined_data.shape

(307511, 27)

In [20]:
combined_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 27 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SK_ID_CURR_APP            307511 non-null  int64  
 1   TARGET                    307511 non-null  int64  
 2   SK_ID_CURR_BUR            177604 non-null  float64
 3   BUR_CNT                   177604 non-null  float64
 4   BUR_ACTIVE_CNT            177604 non-null  float64
 5   BUR_SOLD_CNT              177604 non-null  float64
 6   BUR_BAD_DEPT_CNT          177604 non-null  float64
 7   BUR_CUR_USED              177604 non-null  float64
 8   BUR_CRD_TYPES             177604 non-null  float64
 9   BUR_DAYS_CREDIT_VAR       177604 non-null  float64
 10  BUR_OD_DAY_CNT            177604 non-null  float64
 11  BUR_OD_DAY_VAR            177604 non-null  float64
 12  BUR_DAYS_ED_FACT_VAR      120733 non-null  float64
 13  BUR_OD_AMT_MAX_CNT        177604 non-null  f

In [19]:
combined_data.corrwith('TARGET')

AttributeError: ignored